In [21]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.integrate import quad
from scipy.stats import norm, binom
from scipy.optimize import minimize
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [22]:
df = pd.read_csv('data\SP_historical_PD_data.csv', sep=';')

In [23]:
df.shape

(40, 8)

In [24]:
df.head()

,Year,Total defaults*,Investment-grade defaults,Speculative-grade defaults,Default rate (%),Investment-grade default rate (%),Speculative-grade default rate (%),Total debt outstanding (bil. $)
0,1981,2,0,2,0.15,0.00,0.63,0.06
1,1982,18,2,15,1.22,0.19,4.46,0.90
2,1983,12,1,10,0.77,0.09,2.98,0.37
3,1984,14,2,12,0.93,0.17,3.31,0.36
4,1985,19,0,18,1.13,0.00,4.37,0.31


In [25]:
df["pd_total"] = df["Default rate (%)"] / 100
df["pd_inv"] = df["Investment-grade default rate (%)"] / 100
df["pd_spec"] = df["Speculative-grade default rate (%)"] / 100

df['num_of_inv_grades'] = (df['Investment-grade defaults'] / (df["pd_inv"])).round()
df['num_of_spec_grades'] = (
            df['Speculative-grade defaults'] / (df["pd_spec"])).round().astype(int)
df['num_of_total_grades'] = (df['Total defaults*'] / (df["pd_total"])).round().astype(int)

# Fill-out the missing values in num_of_inv_grades column with the difference between num_of_total_grades and num_of_spec_grades
df['num_of_inv_grades'] = np.where(df['num_of_inv_grades'].isna(), df['num_of_total_grades'] - df['num_of_spec_grades'],
                                   df['num_of_inv_grades']).astype(int)

In [26]:
from scipy.integrate import cumtrapz
from src.variable_change import a_calc_func, b_calc_func, w_calc_func, gamma_calc_func

In [27]:
def calc_linear_likelihood(d_g_arr, n_g_arr, p_g, prob_dens_func, a, b):
    y_values = np.linspace(0, 1, num=1000)

    y_dim = len(y_values)
    a_dim = len(a)

    a_mat = np.tile(a, (y_dim, 1))
    b_mat = np.tile(b, (y_dim, 1))
    y_mat = np.tile(y_values, (a_dim, 1)).T

    integrand_values = np.prod(binom.pmf(d_g_arr, n_g_arr, norm.cdf(a_mat * norm.ppf(y_mat) + b_mat)), axis=1)
    result = cumtrapz(integrand_values, y_values)[-1]
    return result


def log_likehood_variable_changed_fast(d_g_array, n_g_array, p_g, prob_dens_func, a, b):
    return sum(np.log(calc_linear_likelihood(d_g_list, n_g_list, p_g, prob_dens_func, a, b)) for d_g_list, n_g_list in zip(d_g_array, n_g_array))

In [28]:
from src.sucess_probability import p_g

def mle_trapz_g_and_w(
        default_table, num_of_obligors_table, factor_loading_init, gamma_list_init, fixed_w=False, fixed_g=False):

    a_init = np.array(a_calc_func(np.array(factor_loading_init), np.array(gamma_list_init)))
    b_init = np.array(b_calc_func(np.array(factor_loading_init), np.array(gamma_list_init)))

    initial_guess = np.concatenate((a_init, b_init))

    num_of_a = len(a_init)
    bounds = [(-10, 10)] * len(initial_guess)

    # Optimization
    if not fixed_w and not fixed_g:
        objective_function = lambda params: -log_likehood_variable_changed_fast(
            default_table, num_of_obligors_table, p_g, norm.pdf, params[:num_of_a], params[num_of_a:len(initial_guess)]
        )

        result = minimize(objective_function,
                          initial_guess,
                          method="Nelder-Mead",
                          bounds=bounds,
                          options={
                              'disp': False})

        factor_loading_result = np.array(w_calc_func(np.array(result.x[:num_of_a]), np.array(result.x[num_of_a:])))
        gamma_result = np.array(gamma_calc_func(np.array(result.x[:num_of_a]), np.array(result.x[num_of_a:])))

    elif fixed_w:
        objective_function = lambda params: -log_likehood_variable_changed_fast(
            default_table, num_of_obligors_table, p_g, norm.pdf, a_init, params
        )

        result = minimize(objective_function,
                          b_init,
                          method="Nelder-Mead",
                          bounds=bounds[num_of_a:],
                          options={
                              'disp': False})

        factor_loading_result = np.array(w_calc_func(a_init, result.x))
        gamma_result = np.array(gamma_calc_func(a_init, result.x))

    elif fixed_g:
        objective_function = lambda params: -log_likehood_variable_changed_fast(
            default_table, num_of_obligors_table, p_g, norm.pdf, params, b_init
        )

        result = minimize(objective_function,
                          a_init,
                          method="Nelder-Mead",
                          bounds=bounds[:num_of_a],
                          options={
                              'disp': False})

        factor_loading_result = np.array(w_calc_func(result.x, b_init))
        gamma_result = np.array(gamma_calc_func(result.x, b_init))

    return factor_loading_result, gamma_result, result

In [29]:
factor_loading_mle, gamma_mle, mle_result = mle_trapz_g_and_w(df[['Investment-grade defaults', 'Speculative-grade defaults']].values,
                                                              df[['num_of_inv_grades', 'num_of_spec_grades']].values, [0.3, 0.3], [-3, -2.3])

print(f"The factor loading for investment-grade is {factor_loading_mle[0]}, and the gamma parameter is {gamma_mle[0]}")
print(f"The factor loading for speculative-grade is {factor_loading_mle[1]}, and the gamma parameter is {gamma_mle[1]}")

The factor loading for investment-grade is 0.29884580325918636, and the gamma parameter is -3.14207256610641
The factor loading for speculative-grade is 0.2676589012970297, and the gamma parameter is -1.7524064907810293
